### N&auml;chster Punkt finden
Gegeben seien x- und y-Koordinate einer Punktes `pt = (0, 6)` und eine Liste `pts` mit Punkten,  
`pts = [(0, 0), (0, 5), (0, 10)]`.  
Finde den Punkt in `pts`, der am n&auml;chsten bei `pt` liegt.  

1. Erstelle eine Liste mit Paaren der Form `(dist, idx)`,
   wobei `idx` der Index eines Punktes in der Liste ist und `dist` die Distanz von diesem Punkt zum Punkt `pt`.  
   Z.B. `dist_idx = [(6, 0), (1, 1), (4, 2)]` 
1. Finde mit  `min(dist_idx)` das kleinste Tuple dieser Liste (das Tuple `(1, 1)`).  
   Tuple werden in erster Priorit&auml;t nach dem ersten Element (hier die Distanz) sortiert.
1. Das zweite Element dieses Tupels ist der Index des gesuchten Punktes.
   Also der Punkt `pts[1]`, bez. `(0, 5)`.

Oft d&uuml;nnt man die Liste `dist_idx` aus und beh&auml;lt nur Punkte, welche im Umkreis einer Distanz `err` von `pt` liegen.


In [ ]:
#%%file get_closest.py
def distance(p, q):
    return ((p[0] - q[0])**2 + (p[1] - q[1])**2)**0.5


def get_closest(pts, pt0, err=10):
    dist_idx = []
    for i, pt in enumerate(pts):
        dist = distance(pt, pt0)
        if dist < err:
            dist_idx.append((dist, i))

    dist, idx = min(dist_idx, default=(None, None))
    return idx

In [ ]:
pts = [(0, 0), (0, 5), (0, 10)]
pt = (0, 6)
idx = get_closest(pts, pt, err=2)
pts[idx]

### Anwendung: Steine auf Leinwand setzen und verschieben
Siehe auch `MoveStone.ipynb`

In [ ]:
def place_stone(canvas, idx, color='black'):
    x, y = pts[idx]
    canvas.fill_style = color
    canvas.fill_circle(x, y, radius)


def remove_stone(canvas, idx):
    x, y = pts[idx]
    canvas.clear_rect(x-radius-0.5, y-radius-0.5, 2*radius+1)


def move_stone(canvas, src, target, color='black'):
    remove_stone(canvas, src)
    place_stone(canvas, target, color)

In [ ]:
from widgets_helpers import new_output


clicked_point = idx
err_out = new_output()


@err_out.capture()
def on_mouse_down(x, y, canvas):
    global clicked_point
    idx = get_closest(pts, (x, y))
    if idx is not None:
        clicked_point = idx
        place_stone(canvas, idx, colors[idx])


@err_out.capture()
def on_mouse_up(x, y, canvas):
    global clicked_point
    if clicked_point is None:
        return
    target = get_closest(pts, (x, y))
    if target is not None:
        move_stone(canvas, clicked_point, target, colors[target])
        clicked_point = None

In [ ]:
from widgets_helpers import new_mcanvas
from IPython.display import display


pts = [(180, 100), (140, 30), (60, 30), (20, 100), (60, 170), (140, 170)]
colors = ['LightSalmon', 'Salmon', 'Red', 'Crimson', 'FireBrick', 'DarkRed']
radius = 10


mcanvas = new_mcanvas(2, width=200, height=200)
bg, fg = mcanvas

mcanvas.on_mouse_down(lambda x, y, canvas=fg: on_mouse_down(x, y, canvas))
mcanvas.on_mouse_up(lambda x, y, canvas=fg: on_mouse_up(x, y, canvas))


for pt in pts:
    bg.fill_style = 'blue'
    bg.fill_circle(*pt, 3)

display(mcanvas, err_out)